In [ ]:
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import numpy as np
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [9]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [10]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]


In [11]:
# 30 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M30
start_datetime = datetime(2025,1,1)
end_datetime = datetime(2025,2,12)
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
ohlc_df

,time,open,high,low,close
0,2024-12-31 23:00:00,1.03586,1.03610,1.03577,1.03587
1,2024-12-31 23:30:00,1.03587,1.03598,1.03527,1.03527
2,2025-01-02 00:00:00,1.03515,1.03545,1.03487,1.03539
3,2025-01-02 00:30:00,1.03541,1.03579,1.03536,1.03556
4,2025-01-02 01:00:00,1.03556,1.03567,1.03503,1.03515
...,...,...,...,...,...
1388,2025-02-11 21:00:00,1.03456,1.03466,1.03422,1.03462
1389,2025-02-11 21:30:00,1.03462,1.03816,1.03460,1.03584
1390,2025-02-11 22:00:00,1.03584,1.03638,1.03576,1.03622
1391,2025-02-11 22:30:00,1.03622,1.03676,1.03616,1.03631


In [12]:
# hour4 df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_H4
start_datetime = datetime(2025,1,1)
end_datetime = datetime(2025,2,12)
hourly_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
hourly_df

,time,open,high,low,close
0,2025-01-02 00:00:00,1.03515,1.03609,1.03443,1.03579
1,2025-01-02 04:00:00,1.03579,1.03756,1.03578,1.03638
2,2025-01-02 08:00:00,1.03638,1.03725,1.03464,1.03517
3,2025-01-02 12:00:00,1.03517,1.03569,1.03115,1.03134
4,2025-01-02 16:00:00,1.03134,1.03239,1.02243,1.02650
...,...,...,...,...,...
169,2025-02-11 04:00:00,1.03021,1.03060,1.02923,1.03047
170,2025-02-11 08:00:00,1.03047,1.03221,1.02999,1.03180
171,2025-02-11 12:00:00,1.03181,1.03286,1.03145,1.03225
172,2025-02-11 16:00:00,1.03225,1.03513,1.03219,1.03454


In [13]:
# daily df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_D1
start_datetime = datetime(2025,1,1)
end_datetime = datetime(2025,2,12)
daily_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
daily_df

,time,open,high,low,close
0,2025-01-02,1.03515,1.03756,1.02243,1.02670
1,2025-01-03,1.02572,1.03099,1.02529,1.03080
2,2025-01-06,1.03021,1.04369,1.02955,1.03897
3,2025-01-07,1.03896,1.04345,1.03396,1.03398
4,2025-01-08,1.03401,1.03577,1.02732,1.03191
5,2025-01-09,1.03181,1.03216,1.02836,1.02994
6,2025-01-10,1.02989,1.03119,1.02134,1.02424
7,2025-01-13,1.02236,1.02499,1.01773,1.02424
8,2025-01-14,1.02443,1.03084,1.02379,1.03077
9,2025-01-15,1.03047,1.03544,1.02593,1.02902


In [14]:
# start b2 strategy
# Calculate Pivot Point, R1, R2, R3, stopeR4, S1, S2, S3, stopeS4 levels with Fibonacci retracement
daily_df["pivot"] = (daily_df["high"] + daily_df["low"] + daily_df["close"]) / 3
daily_df["prange"] = (daily_df["high"] - daily_df["low"])
daily_df["R1"] = daily_df["pivot"] + 0.382 * daily_df["prange"]
daily_df["R2"] = daily_df["pivot"] + 0.618 * daily_df["prange"]
daily_df["R3"] = daily_df["pivot"] + 1.000 * daily_df["prange"]

daily_df["S1"] = daily_df["pivot"] - 0.382 * daily_df["prange"]
daily_df["S2"] = daily_df["pivot"] - 0.618 * daily_df["prange"]
daily_df["S3"] = daily_df["pivot"] - 1.000 * daily_df["prange"]
daily_df

,time,open,high,low,close,pivot,prange,R1,R2,R3,S1,S2,S3
0,2025-01-02,1.03515,1.03756,1.02243,1.02670,1.028897,0.01513,1.034676,1.038247,1.044027,1.023117,1.019546,1.013767
1,2025-01-03,1.02572,1.03099,1.02529,1.03080,1.029027,0.00570,1.031204,1.032549,1.034727,1.026849,1.025504,1.023327
2,2025-01-06,1.03021,1.04369,1.02955,1.03897,1.037403,0.01414,1.042805,1.046142,1.051543,1.032002,1.028665,1.023263
3,2025-01-07,1.03896,1.04345,1.03396,1.03398,1.037130,0.00949,1.040755,1.042995,1.046620,1.033505,1.031265,1.027640
4,2025-01-08,1.03401,1.03577,1.02732,1.03191,1.031667,0.00845,1.034895,1.036889,1.040117,1.028439,1.026445,1.023217
5,2025-01-09,1.03181,1.03216,1.02836,1.02994,1.030153,0.00380,1.031605,1.032502,1.033953,1.028702,1.027805,1.026353
6,2025-01-10,1.02989,1.03119,1.02134,1.02424,1.025590,0.00985,1.029353,1.031677,1.035440,1.021827,1.019503,1.015740
7,2025-01-13,1.02236,1.02499,1.01773,1.02424,1.022320,0.00726,1.025093,1.026807,1.029580,1.019547,1.017833,1.015060
8,2025-01-14,1.02443,1.03084,1.02379,1.03077,1.028467,0.00705,1.031160,1.032824,1.035517,1.025774,1.024110,1.021417
9,2025-01-15,1.03047,1.03544,1.02593,1.02902,1.030130,0.00951,1.033763,1.036007,1.039640,1.026497,1.024253,1.020620


In [17]:
#setting the sma 1 and 14 periods
furst_sma_period = 1
second_sma_period = 14
# sma 1
hourly_df['sma_1'] = hourly_df['close'].rolling(furst_sma_period).mean()
hourly_df['prev_sma_1'] = hourly_df['sma_1'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 14
hourly_df['sma_14'] = hourly_df['close'].rolling(second_sma_period).mean()
hourly_df

,time,open,high,low,close,sma_1,prev_sma_1,sma_14
0,2025-01-02 00:00:00,1.03515,1.03609,1.03443,1.03579,1.03579,NaN,NaN
1,2025-01-02 04:00:00,1.03579,1.03756,1.03578,1.03638,1.03638,1.03579,NaN
2,2025-01-02 08:00:00,1.03638,1.03725,1.03464,1.03517,1.03517,1.03638,NaN
3,2025-01-02 12:00:00,1.03517,1.03569,1.03115,1.03134,1.03134,1.03517,NaN
4,2025-01-02 16:00:00,1.03134,1.03239,1.02243,1.02650,1.02650,1.03134,NaN
...,...,...,...,...,...,...,...,...
169,2025-02-11 04:00:00,1.03021,1.03060,1.02923,1.03047,1.03047,1.03021,1.033269
170,2025-02-11 08:00:00,1.03047,1.03221,1.02999,1.03180,1.03180,1.03047,1.032798
171,2025-02-11 12:00:00,1.03181,1.03286,1.03145,1.03225,1.03225,1.03180,1.032421
172,2025-02-11 16:00:00,1.03225,1.03513,1.03219,1.03454,1.03454,1.03225,1.032094


In [19]:
#setting the 11 and 114 sma periods
thurd_sma_period = 1
fourth_sma_period = 14
# sma 11
daily_df['sma_11'] = daily_df['close'].rolling(thurd_sma_period).mean()
daily_df['prev_sma_11'] = daily_df['sma_11'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 114
daily_df['sma_114'] = daily_df['close'].rolling(fourth_sma_period).mean()
daily_df

,time,open,high,low,close,pivot,prange,R1,R2,R3,S1,S2,S3,sma_11,prev_sma_11,sma_114
0,2025-01-02,1.03515,1.03756,1.02243,1.02670,1.028897,0.01513,1.034676,1.038247,1.044027,1.023117,1.019546,1.013767,1.02670,NaN,NaN
1,2025-01-03,1.02572,1.03099,1.02529,1.03080,1.029027,0.00570,1.031204,1.032549,1.034727,1.026849,1.025504,1.023327,1.03080,1.02670,NaN
2,2025-01-06,1.03021,1.04369,1.02955,1.03897,1.037403,0.01414,1.042805,1.046142,1.051543,1.032002,1.028665,1.023263,1.03897,1.03080,NaN
3,2025-01-07,1.03896,1.04345,1.03396,1.03398,1.037130,0.00949,1.040755,1.042995,1.046620,1.033505,1.031265,1.027640,1.03398,1.03897,NaN
4,2025-01-08,1.03401,1.03577,1.02732,1.03191,1.031667,0.00845,1.034895,1.036889,1.040117,1.028439,1.026445,1.023217,1.03191,1.03398,NaN
5,2025-01-09,1.03181,1.03216,1.02836,1.02994,1.030153,0.00380,1.031605,1.032502,1.033953,1.028702,1.027805,1.026353,1.02994,1.03191,NaN
6,2025-01-10,1.02989,1.03119,1.02134,1.02424,1.025590,0.00985,1.029353,1.031677,1.035440,1.021827,1.019503,1.015740,1.02424,1.02994,NaN
7,2025-01-13,1.02236,1.02499,1.01773,1.02424,1.022320,0.00726,1.025093,1.026807,1.029580,1.019547,1.017833,1.015060,1.02424,1.02424,NaN
8,2025-01-14,1.02443,1.03084,1.02379,1.03077,1.028467,0.00705,1.031160,1.032824,1.035517,1.025774,1.024110,1.021417,1.03077,1.02424,NaN
9,2025-01-15,1.03047,1.03544,1.02593,1.02902,1.030130,0.00951,1.033763,1.036007,1.039640,1.026497,1.024253,1.020620,1.02902,1.03077,NaN


In [29]:
#setting entring and closing time and one trade a day
# Set the start and end times for your desired trading hours
start_time = datetime.strptime('06:00:00', '%H:%M:%S').time()
end_time = datetime.strptime('15:00:00', '%H:%M:%S').time()

# Filter the trade data for trades within the specified trading hours
trades_within_hours = daily_df[(daily_df['time'].dt.time >= start_time) & (daily_df['time'].dt.time <= end_time)]

# Close all open positions at 11:45 PM
first_trade_per_day = daily_df.groupby(daily_df['time'].dt.time).first()
end_trading_time = datetime.strptime('23:45:00', '%H:%M:%S').time()
open_positions = trades_within_hours[trades_within_hours['time'].dt.time == end_trading_time]
# function to trading
daily_df["sell_stope_loss"] = daily_df["high"] >= daily_df["R2"] + 0.0001*40
daily_df["bearish_pivot"] = daily_df["high"] >= daily_df["R2"]
daily_df["buy_stope_loss"] = daily_df["low"] <= daily_df["S2"] - 0.0001*40
daily_df["bullish_pivot"] = daily_df["low"] <= daily_df["S2"]
daily_df['sell_profit'] = daily_df['low'] <= daily_df["R1"] - daily_df['R3'] + daily_df["R1"] - 0.0001*40
daily_df['buy_profit'] = daily_df['high'] >= daily_df["S1"] - daily_df["S3"] + daily_df["S1"] + 0.0001*40
daily_df

,time,open,high,low,close,pivot,prange,R1,R2,R3,...,S3,sma_11,prev_sma_11,sma_114,sell_stope_loss,bearish_pivot,buy_stope_loss,bullish_pivot,sell_profit,buy_profit
0,2025-01-02,1.03515,1.03756,1.02243,1.02670,1.028897,0.01513,1.034676,1.038247,1.044027,...,1.013767,1.02670,NaN,NaN,False,False,False,False,False,True
1,2025-01-03,1.02572,1.03099,1.02529,1.03080,1.029027,0.00570,1.031204,1.032549,1.034727,...,1.023327,1.03080,1.02670,NaN,False,False,False,True,False,False
2,2025-01-06,1.03021,1.04369,1.02955,1.03897,1.037403,0.01414,1.042805,1.046142,1.051543,...,1.023263,1.03897,1.03080,NaN,False,False,False,False,True,False
3,2025-01-07,1.03896,1.04345,1.03396,1.03398,1.037130,0.00949,1.040755,1.042995,1.046620,...,1.027640,1.03398,1.03897,NaN,False,True,False,False,False,True
4,2025-01-08,1.03401,1.03577,1.02732,1.03191,1.031667,0.00845,1.034895,1.036889,1.040117,...,1.023217,1.03191,1.03398,NaN,False,False,False,False,False,False
5,2025-01-09,1.03181,1.03216,1.02836,1.02994,1.030153,0.00380,1.031605,1.032502,1.033953,...,1.026353,1.02994,1.03191,NaN,False,False,False,False,False,False
6,2025-01-10,1.02989,1.03119,1.02134,1.02424,1.025590,0.00985,1.029353,1.031677,1.035440,...,1.015740,1.02424,1.02994,NaN,False,False,False,False,False,False
7,2025-01-13,1.02236,1.02499,1.01773,1.02424,1.022320,0.00726,1.025093,1.026807,1.029580,...,1.015060,1.02424,1.02424,NaN,False,False,False,True,False,False
8,2025-01-14,1.02443,1.03084,1.02379,1.03077,1.028467,0.00705,1.031160,1.032824,1.035517,...,1.021417,1.03077,1.02424,NaN,False,False,False,True,False,False
9,2025-01-15,1.03047,1.03544,1.02593,1.02902,1.030130,0.00951,1.033763,1.036007,1.039640,...,1.020620,1.02902,1.03077,NaN,False,False,False,False,False,False


In [ ]:
def entring_stoploss_profit (row) :
    bullish_crossover = row['sma_1'] > row['sma_14'] and row['sma_11'] > row['sma_114'] and row['prev_sma_1'] < row['sma_14'] and row['prev_sma_11'] < row['sma_114']
    bearish_crossover = row['sma_1'] < row['sma_14'] and row['sma_11'] < row['sma_114'] and row['prev_sma_1'] > row['sma_14'] and row['prev_sma_11'] > row['sma_114']
    buy_signals = bullish_crossover and row["bullish_pivot"] and open_positions and first_trade_per_day 
    sell_signals = bearish_crossover and row["bearish_pivot"] and open_positions and first_trade_per_day
    close_condition1 = row["stopeS4"] or end_trading_time
    close_condition2 = row["stopeR4"] or end_trading_time
    if buy_signals == True :
        return "buy"
    else :
        return False
    
    if sell_signals :
        return True
        
    else :
        return False

ohlc_df['entring_stoploss_profit'] = ohlc_df.apply(entring_stoploss_profit, axis=1)
ohlc_df

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [27]:
# visualizing the ohlc data
fig = go.Figure(data = [go.Candlestick(x = ohlc_df['time'],
                                       open = ohlc_df['open'],
                                       high = ohlc_df['high'],
                                       low = ohlc_df['low'],
                                       close = ohlc_df['close'])])

# fig.update_layout(height=600, xaxis_rangeslider_visible=False)
# 📌 Add Pivot Points as Horizontal Lines
pivot_levels = ["R1", "R2", "S1", "S2"]
colors = {'R1': 'red', 'R2': 'red', 'S1': 'green', 'S2': 'green'}


for _, row in daily_df.iterrows():
    for level in pivot_levels:
        fig.add_trace(go.Scatter(
            x=[row['time'], row['time'] + pd.Timedelta(days=1)],  # Extend for one day
            y=[row[level], row[level]],
            mode='lines',
            name=f"{level} {row['time']}",
            line=dict(color=colors[level], width=1.5, dash='dot')
        ))

# 📌 Customize Layout
fig.update_layout(
    title="Pivot Points on Candlestick Chart",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis_rangeslider_visible=False
)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'close': {'bdata': ('NIXOa+yS8D+NeohGd5DwP+KvyRr1kP' ... 'dsW5TwP79IaMu5lPA/Tpzc71CU8D8='),
                        'dtype': 'f8'},
              'high': {'bdata': ('ayv2l92T8D8W9rTDX5PwP4xK6gQ0kf' ... 'MxA5XwP7vQXKeRlvA/MPXzpiKV8D8='),
                       'dtype': 'f8'},
              'low': {'bdata': ('w9hCkIOS8D+NeohGd5DwP8nIWdjTjv' ... 'gTeZLwPxXGFoIclPA/Ft7lIr6T8D8='),
                      'dtype': 'f8'},
              'open': {'bdata': ('wsBz7+GS8D80hc5r7JLwPzlFR3L5j/' ... '72zJLwP79gN2xblPA/TYQNT6+U8D8='),
                       'dtype': 'f8'},
              'type': 'candlestick',
              'x': array(['2024-12-31T23:00:00.000000000', '2024-12-31T23:30:00.000000000',
                          '2025-01-02T00:00:00.000000000', ..., '2025-02-11T22:00:00.000000000',
                          '2025-02-11T22:30:00.000000000', '2025-02-11T23:00:00.000000000'],
                         shape=(1393,), dtype='datetime64[ns]')},
             {'line': {'color': 'red', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'R1 2025-01-02 00:00:00',
              'type': 'scatter',
              'x': [2025-01-02 00:00:00, 2025-01-03 00:00:00],
              'y': [1.0346763266666665, 1.0346763266666665]},
             {'line': {'color': 'red', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'R2 2025-01-02 00:00:00',
              'type': 'scatter',
              'x': [2025-01-02 00:00:00, 2025-01-03 00:00:00],
              'y': [1.0382470066666667, 1.0382470066666667]},
             {'line': {'color': 'green', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'S1 2025-01-02 00:00:00',
              'type': 'scatter',
              'x': [2025-01-02 00:00:00, 2025-01-03 00:00:00],
              'y': [1.0231170066666666, 1.0231170066666666]},
             {'line': {'color': 'green', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'S2 2025-01-02 00:00:00',
              'type': 'scatter',
              'x': [2025-01-02 00:00:00, 2025-01-03 00:00:00],
              'y': [1.0195463266666664, 1.0195463266666664]},
             {'line': {'color': 'red', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'R1 2025-01-03 00:00:00',
              'type': 'scatter',
              'x': [2025-01-03 00:00:00, 2025-01-04 00:00:00],
              'y': [1.0312040666666669, 1.0312040666666669]},
             {'line': {'color': 'red', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'R2 2025-01-03 00:00:00',
              'type': 'scatter',
              'x': [2025-01-03 00:00:00, 2025-01-04 00:00:00],
              'y': [1.0325492666666667, 1.0325492666666667]},
             {'line': {'color': 'green', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'S1 2025-01-03 00:00:00',
              'type': 'scatter',
              'x': [2025-01-03 00:00:00, 2025-01-04 00:00:00],
              'y': [1.0268492666666666, 1.0268492666666666]},
             {'line': {'color': 'green', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'S2 2025-01-03 00:00:00',
              'type': 'scatter',
              'x': [2025-01-03 00:00:00, 2025-01-04 00:00:00],
              'y': [1.0255040666666668, 1.0255040666666668]},
             {'line': {'color': 'red', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'R1 2025-01-06 00:00:00',
              'type': 'scatter',
              'x': [2025-01-06 00:00:00, 2025-01-07 00:00:00],
              'y': [1.0428048133333334, 1.0428048133333334]},
             {'line': {'color': 'red', 'dash': 'dot', 'width': 1.5},
              'mode': 'lines',
              'name': 'R2 2025-01-06 00:00:00',
              'type': 'scatter',
              'x': [2025-01-06 00:00:00, 2025-01-07 00:00:00],
              'y': [1.04